In [1]:
# Instalar dependencias (si no lo tienes aún)
!pip install langchain langchain_community chromadb pdfplumber
!pip install -U langchain-community pypdf
!pip install -U langchain-ollama langchain_chroma


  Using cached langchain-1.1.3-py3-none-any.whl (102 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
  Using cached chromadb-1.3.6-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.6 MB)
  Using cached pdfplumber-0.11.8-py3-none-any.whl (60 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl (463 kB)
  Using cached langchain_core-1.1.3-py3-none-any.whl (475 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl (157 kB)
  Using cached langsmith-0.4.58-py3-none-any.whl (412 kB)
  Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Using cached aiohttp-3.13.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.7 MB)
  Using cached langchain_classic-1.0.0-py3-none-any.whl (1.0 MB)
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl (51 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.ma

In [2]:
# Importar librerías
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
import chromadb

In [ ]:
# Cargar documentos
# Puedes cargar PDFs o TXT
pdf_loader = DirectoryLoader("../documentos/pdfs/manuales", glob="*.pdf", loader_cls=PyPDFLoader)  # carpeta con PDFs
# txt_loader = DirectoryLoader("documentos", glob="*.txt", loader_cls=TextLoader)

# docs = pdf_loader.load() + txt_loader.load()
docs = pdf_loader.load()

# Dividir en fragmentos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

Multiple definitions in dictionary at byte 0xc59 for key /x10
Multiple definitions in dictionary at byte 0xc71 for key /x10


In [18]:
# Crear embeddings usando Ollama
# Se asume que Ollama está corriendo en otro servidor y accesible
embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b", base_url="http://localhost:11434")  

In [19]:
# Ensure the host includes the scheme (http) — otherwise the HTTP path constructed by the client
# can be incorrect and produce a 404 HTML response.
client = chromadb.HttpClient(host="localhost", port=3000, ssl=False)

# Conectar al Chroma remoto
vector_store = Chroma(
    client=client,
    embedding_function=embeddings,
    collection_name="qa_servicio_internet_manuales",
)

# Indexar los fragmentos (chunks ya está definido en celdas anteriores)
# Envolvemos en try/except para capturar errores de conexión/HTTP y mostrarlos claramente.
try:
    vector_store.add_documents(chunks)
    print("Vector store creado y poblado en Chroma DB remoto.")
except ValueError as e:
    # Si el servidor responde con HTML 404 u otro error, lo mostramos para depuración
    print("Error al indexar en Chroma remota:", e)
    raise

Vector store creado y poblado en Chroma DB remoto.


In [21]:
query = "fijate que mi router no enciende"
results = vector_store.similarity_search(query, k=5)

for r in results:
    print(r.page_content)

2. Coloca el modem/router en un lugar abierto y céntrico de la casa. Donde pusieras una bocina
si quisieras oírla en toda la casa.
En casas muy grandes o de dos plantas recomendamos la utilización de un Amplificador de Wi-Fi
o repetidor. Su ubicación debe ser donde la señal del modem esta en -60dbm.
 
formas incorrectas de colocar tu modem (Wi-Fi):
 
1. Cerca de teléfonos inalámbricos de 2.4GHZ, microondas, radios de bebes, cámaras
inalámbricas. Estos equipos normalmente transmiten en la misma frecuencia (2.4 GHZ) que los
router lo que provoca interferencias y bajo rendimiento de la red. Evitar poner modem cerca de
las cocinas.
 
2. Cerca de ventanas o puertas que den al exterior. Las ondas se pierden y se debilitan en el
interior cuando parte de estas se fugan para el exterior.
 
3. Dentro de muebles o estantes, los mismos impiden que las ondas se esparzan y lleguen donde
se requiere.
 
4. Evitar la mezcla de alimentación de los cables de DSL y electricidad. Esto provoca
atenuación po